In [2]:
using Pkg
Pkg.activate(".")

using DataFrames
using NemoMod
using SQLite
using Cbc
using JuMP

include("setup_analysis.jl")

  Activating project at `~/Documents/Projects/git_jbus/lac_decarbonization/julia`


"/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite"

In [3]:
ispath(fp_sqlite_nemomod_db_tmp)

true

In [32]:
!ispath(dir_tmp) ? mkdir(dir_tmp) : nothing
fp_db = joinpath(dir_tmp, "testingdb.sqlite")
#createnemodb(fp_db)

2022-14-Aug 13:49:56.453 Opened SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.
2022-14-Aug 13:49:56.684 Added NEMO structure to SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.


SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite")

In [31]:
rm(fp_db)
createnemodb(fp_db)

2022-14-Aug 13:18:02.188 Opened SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.
2022-14-Aug 13:18:02.267 Added NEMO structure to SQLite database at /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite.


SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite")

In [10]:

db2 = SQLite.DB(fp_db)

dict_tables2 = Dict{String, Dat aFrame}()
tables_vary2 = []

tabs = SQLite.tables(db2)[1]
table_name = 0
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db2, "select * from $(table_name)") |> DataFrame
    dict_tables2[table_name] = df_tmp
    (nrow(df_tmp) > 0) ? push!(tables_vary2, table_name) : nothing
end



In [29]:
df_out = DBInterface.execute(db2, "select sql from sqlite_master where type = 'index';")



SQLite.Query(SQLite.Stmt(SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/testingdb.sqlite"), 94), Base.RefValue{Int32}(100), [:sql], Type[Union{Missing, String}], Dict(:sql => 1), Base.RefValue{Int64}(0))

### Try running the model

In [11]:
@time NemoMod.calculatescenario(
    fp_sqlite_nemomod_db_tmp; 
    jumpmodel = Model(Cbc.Optimizer),
    numprocs = 1
)

2022-14-Aug 18:47:37.208 Started modeling scenario.
2022-14-Aug 18:47:37.215 Validated run-time arguments.
2022-14-Aug 18:47:37.224 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite.
2022-14-Aug 18:47:37.257 Dropped pre-existing result tables from database.
2022-14-Aug 18:47:37.258 Verified that transmission modeling is not enabled.
2022-14-Aug 18:47:37.366 Created parameter views and indices.
2022-14-Aug 18:47:37.380 Created temporary tables.
2022-14-Aug 18:47:37.680 Executed core database queries.
2022-14-Aug 18:47:37.692 Defined dimensions.
2022-14-Aug 18:47:37.696 Defined demand variables.
2022-14-Aug 18:47:37.702 Defined storage variables.
2022-14-Aug 18:47:37.703 Defined capacity variables.
2022-14-Aug 18:47:37.879 Defined activity variables.
2022-14-Aug 18:47:37.880 Defined costing variables.
2022-14-Aug 18:47:37.880 Defined reserve margin variables.
2022-14-Aug 18:47:37.882 Defined emissi

INFEASIBLE_OR_UNBOUNDED::TerminationStatusCode = 6

In [64]:
filter(x -> occursin("Emission", x), keys(dict_tables_demo))

Set{String} with 6 elements:
  "ModelPeriodEmissionLimit"
  "ModelPeriodExogenousEmission"
  "AnnualEmissionLimit"
  "EmissionsPenalty"
  "AnnualExogenousEmission"
  "EmissionActivityRatio"

In [62]:
?NemoMod.calculatescenario

```
calculatescenario(dbpath::String; jumpmodel::JuMP.Model = Model(Cbc.Optimizer),
    calcyears::Array{Int, 1} = Array{Int, 1}(),
    varstosave::String = "vdemandnn, vnewcapacity, vtotalcapacityannual,
        vproductionbytechnologyannual, vproductionnn, vusebytechnologyannual,
        vusenn, vtotaldiscountedcost",
    restrictvars::Bool = true, reportzeros::Bool = false,
    continuoustransmission::Bool = false,
    forcemip::Bool = false, startvalsdbpath::String = "",
    startvalsvars::String = "", precalcresultspath::String = "",
    quiet::Bool = false
)
```

Calculates a scenario specified in a scenario database. Returns a `MathOptInterface.TerminationStatusCode` indicating the termination status reported by the solver used for the calculation (e.g., `OPTIMAL::TerminationStatusCode = 1`).

# Arguments

  * `dbpath::String`: Path to the scenario database, which must be a SQLite version 3 database that   implements NEMO's scenario database structure. See NEMO's documentation on scenario databases   for details. Empty scenario databases can be generated with NEMO's `createnemodb` function.
  * `jumpmodel::JuMP.Model`: [JuMP](https://github.com/jump-dev/JuMP.jl) model object   specifying the solver to be used for the calculation.   Examples: `Model(optimizer_with_attributes(GLPK.Optimizer, "presolve" => true))`,   `Model(CPLEX.Optimizer)`, `Model(optimizer_with_attributes(Gurobi.Optimizer, "NumericFocus" => 1))`.   Note that the solver's Julia package (Julia wrapper) must be installed. See the   documentation for JuMP for information on how to specify a solver and set solver options.
  * `calcyears::Array{Int, 1}`: Years to include in the calculation (a subset of the years specified in   the scenario database). All years in the database are included if this argument is omitted.
  * `varstosave::String`: Comma-delimited list of output variables whose results should be   saved in the scenario database when the scenario is calculated. See NEMO's documentation on   outputs for information on the variables that are available.
  * `restrictvars::Bool`: Indicates whether NEMO should conduct additional data analysis   to limit the set of variables created in the optimization problem for the scenario.   By default, to improve performance, NEMO selectively creates certain variables to   avoid combinations of subscripts that do not exist in the scenario's data. This option   increases the stringency of this filtering. It requires more processing time as the   problem is built, but it can substantially reduce the solve time for large models.
  * `reportzeros::Bool`: Indicates whether results saved in the scenario database should   include values equal to zero. Specifying `false` can substantially improve the   performance of large models.
  * `continuoustransmission::Bool`: Indicates whether continuous (`true`) or binary (`false`)   variables are used to represent investment decisions for candidate transmission lines. Not   relevant in scenarios that do not model transmission.
  * `forcemip::Bool`: Forces NEMO to formulate the optimization problem for the scenario as a   mixed-integer problem. This can improve performance with some solvers (e.g., CPLEX, Mosek). If   this option is set to `false`, the input parameters for the scenario (i.e., in the scenario   database) determine whether the optimization problem is mixed-integer.
  * `startvalsdbpath::String`: Path to a previously calculated scenario database from which NEMO   should take starting values for variables in the optimization problem formulated in this   function. This argument is used in conjunction with `startvalsvars`.
  * `startvalsvars::String`: Comma-delimited list of variables for which starting values should be set.   See NEMO's documentation on outputs for information on the variables that are available. NEMO   takes starting values from output variable results saved in the database identified by   `startvalsdbpath`. Saved results are matched to variables in the optimization problem using   the variables' subscripts, and starting values are set with JuMP's `set_start_value` function.   If `startvalsvars` is an empty string, NEMO sets starting values for all variables present in   both the optimization problem and the `startvalsdbpath` database.
  * `precalcresultspath::String`: Path to a previously calculated scenario database that NEMO should   copy over the database specified by `dbpath`. This argument can also be a directory containing   previously calculated scenario databases, in which case NEMO copies any file in the directory   with the same name as the `dbpath` database. The intent of the argument is to short-circuit   calculations in situations where valid results already exist.
  * `quiet::Bool`: Suppresses low-priority status messages (which are otherwise printed to   `STDOUT`).


In [41]:
dbpath = normpath(joinpath(pathof(NemoMod), "..", "..", "test", "ramp_test.sqlite"))
chmod(dbpath, 0o777)
NemoMod.calculatescenario(dbpath; jumpmodel=Model(Cbc.Optimizer), numprocs = 1)

2022-14-Aug 22:22:33.992 Started modeling scenario.
2022-14-Aug 22:22:33.996 Validated run-time arguments.
2022-14-Aug 22:22:34.007 Connected to scenario database. Path = /Users/jsyme/.julia/packages/NemoMod/vg6Tx/test/ramp_test.sqlite.
2022-14-Aug 22:22:34.049 Dropped pre-existing result tables from database.
2022-14-Aug 22:22:34.051 Verified that transmission modeling is not enabled.
2022-14-Aug 22:22:34.211 Created parameter views and indices.
2022-14-Aug 22:22:34.219 Created temporary tables.
2022-14-Aug 22:22:34.974 Executed core database queries.
2022-14-Aug 22:22:34.988 Defined dimensions.
2022-14-Aug 22:22:35.000 Defined demand variables.
2022-14-Aug 22:22:35.014 Defined storage variables.
2022-14-Aug 22:22:35.017 Defined capacity variables.
2022-14-Aug 22:22:36.505 Defined activity variables.
2022-14-Aug 22:22:36.506 Defined costing variables.
2022-14-Aug 22:22:36.506 Defined reserve margin variables.
2022-14-Aug 22:22:36.506 Defined emissions variables.
2022-14-Aug 22:22:36.5

OPTIMAL::TerminationStatusCode = 1

In [46]:
# get tables from SISEPUEDE
db = SQLite.DB(fp_sqlite_nemomod_db_tmp)

dict_tables = Dict{String, DataFrame}()
tables_vary = []

tabs = SQLite.tables(db)[1]
table_name = 0
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
    dict_tables[table_name] = df_tmp
    (nrow(df_tmp) > 0) ? push!(tables_vary, table_name) : nothing
end


# get tables from DEMO
db_demo = SQLite.DB(dbpath)

dict_tables_demo = Dict{String, DataFrame}()
tables_vary_demo = []

tabs_demo = SQLite.tables(db_demo)[1]
for k in 1:length(tabs_demo)
    table_name = tabs_demo[k]
    df_tmp = DBInterface.execute(db_demo, "select * from $(table_name)") |> DataFrame
    dict_tables_demo[table_name] = df_tmp
    (nrow(df_tmp) > 0) ? push!(tables_vary_demo, table_name) : nothing
end

In [10]:
#setdiff(keys(dict_tables_demo), keys(dict_tables))
for k in keys(dict_tables_demo)
    #(("f" in names(dict_tables_demo[k])) & (nrow(dict_tables_demo[k]) > 0)) ? println(k) : nothing
    if k in keys(dict_tables)
        if (nrow(dict_tables_demo[k]) != 0) & (nrow(dict_tables[k]) == 0)
            println(k)
        end
    end
        
end
#dict_tables_demo["InputActivityRatio"]

StorageLevelStart
StorageFullLoadHours


In [19]:
unique(dict_tables_demo["OutputActivityRatio"][:, :f])

filter(x -> (x[:y] == "2025"), dict_tables_demo["OutputActivityRatio"])

,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,6,1,gas,electricity,generate,2025,1.0
2,16,1,solar,electricity,generate,2025,1.0
3,26,1,storage1,electricity,generate,2025,1.0
4,36,1,gassupply,gas,generate,2025,1.0
5,46,1,solarsupply,solar,generate,2025,1.0


In [47]:
for k in keys(dict_tables_demo)
    
    if "t" in names(dict_tables_demo[k])
        (("gasssupply" in dict_tables_demo[k][:, :t]) | ("solarsupply" in dict_tables_demo[k][:, :t])) ? println(k) : nothing
    end
end

OperationalLife
OutputActivityRatio
vproductionbytechnologyannual
vnewcapacity
vtotalcapacityannual


In [51]:
unique(dict_tables_demo["OutputActivityRatio"][:, :t])

7-element Vector{String}:
 "gas"
 "solar"
 "storage1"
 "gassupply"
 "solarsupply"
 "coal"
 "coalsupply"

In [57]:
filter(z -> (z[:t] == "coal"), dict_tables_demo["OutputActivityRatio"])

,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,51,1,coal,electricity,generate,2020,1.0
2,52,1,coal,electricity,generate,2021,1.0
3,53,1,coal,electricity,generate,2022,1.0
4,54,1,coal,electricity,generate,2023,1.0
5,55,1,coal,electricity,generate,2024,1.0
6,56,1,coal,electricity,generate,2025,1.0
7,57,1,coal,electricity,generate,2026,1.0
8,58,1,coal,electricity,generate,2027,1.0
9,59,1,coal,electricity,generate,2028,1.0


In [61]:
 filter(x -> (x[:t] == "coalsupply"), dict_tables_demo["InputActivityRatio"])

,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64


In [32]:
readdir(normpath(joinpath(pathof(NemoMod), "..", "..", "test")))

12-element Vector{String}:
 "Manifest.toml"
 "Project.toml"
 "cbc_tests.jl"
 "cplex_tests.jl"
 "glpk_tests.jl"
 "gurobi_tests.jl"
 "mosek_tests.jl"
 "ramp_test.sqlite"
 "runtests.jl"
 "storage_test.sqlite"
 "storage_transmission_test.sqlite"
 "xpress_tests.jl"